In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame( [
                    ['Java', 'Alice', 66, 77, 88],
                    ['Python','Alice',56, 77, 88],
                    ['Java', 'Bob', 89, 53, 56],
                    ['Python','Bob',56, 88, 99],
                    ['Java', 'Bo', 22, 56, 66],
                    ['Python','Sam',89, 99, 88]
                    ], columns = ['class', 'student', 'exam1', 'exam2', 'exam3'])
df

,class,student,exam1,exam2,exam3
0,Java,Alice,66,77,88
1,Python,Alice,56,77,88
2,Java,Bob,89,53,56
3,Python,Bob,56,88,99
4,Java,Bo,22,56,66
5,Python,Sam,89,99,88


The final grade is the average of the best two exam
1. What is the final grade of each student
- What is the average final grade of each class
- What students did better than the average in each class
- What students did better than the average in both classes

## 1

In [9]:
#1 approach-1 write a function and apply it on the data frame
def score(list_of_numbers):
    return ( sum(list_of_numbers) - min(list_of_numbers) ) /  ( len(list_of_numbers) - 1)

In [12]:
#because applying on a data frame, we need to sepcify the axis
df['score'] = df[['exam1', 'exam2', 'exam3']].apply(score, axis=1)
df

,class,student,exam1,exam2,exam3,score
0,Java,Alice,66,77,88,82.5
1,Python,Alice,56,77,88,82.5
2,Java,Bob,89,53,56,72.5
3,Python,Bob,56,88,99,93.5
4,Java,Bo,22,56,66,61.0
5,Python,Sam,89,99,88,94.0


In [21]:
# approach-2, operate of the columns as series

In [15]:
df['total'] = df['exam1'] + df['exam2'] + df['exam3']

In [18]:
# next we need to calculate the minimum grade of each row
df['min'] = df[['exam1', 'exam2', 'exam3']].T.min()

In [19]:
df['score'] = (df['total'] - df['min']) / 2

In [20]:
df

,class,student,exam1,exam2,exam3,score,total,min
0,Java,Alice,66,77,88,82.5,231,66
1,Python,Alice,56,77,88,82.5,221,56
2,Java,Bob,89,53,56,72.5,198,53
3,Python,Bob,56,88,99,93.5,243,56
4,Java,Bo,22,56,66,61.0,144,22
5,Python,Sam,89,99,88,94.0,276,88


# 2

In [23]:
course_avg = df.groupby('class')['score'].mean()
course_avg

class
Java      72.0
Python    90.0
Name: score, dtype: float64

# 3

In [28]:
# approach-1, transform score by comparing it to the group mean
# this will return a mask
mask = df.groupby('class')['score'].transform(lambda x: x > x.mean())
mask

0     True
1    False
2     True
3     True
4    False
5     True
Name: score, dtype: bool

In [29]:
df[mask]

,class,student,exam1,exam2,exam3,score,total,min
0,Java,Alice,66,77,88,82.5,231,66
2,Java,Bob,89,53,56,72.5,198,53
3,Python,Bob,56,88,99,93.5,243,56
5,Python,Sam,89,99,88,94.0,276,88


In [27]:
# if we want just names of students
set(df[mask].student)

{'Alice', 'Bob', 'Sam'}

In [35]:
# approach-2, calculate group mean and join it with the original data
# then select when score > group mean
avg_score = df.groupby('class')['score'].mean()
avg_score

class
Java      72.0
Python    90.0
Name: score, dtype: float64

In [39]:
new_data = df.set_index('class').join(avg_score, rsuffix='_avg')
new_data

,student,exam1,exam2,exam3,score,total,min,better_than_average,score_avg
class,,,,,,,,,
Java,Alice,66,77,88,82.5,231,66,True,72.0
Java,Bob,89,53,56,72.5,198,53,True,72.0
Java,Bo,22,56,66,61.0,144,22,False,72.0
Python,Alice,56,77,88,82.5,221,56,False,90.0
Python,Bob,56,88,99,93.5,243,56,True,90.0
Python,Sam,89,99,88,94.0,276,88,True,90.0


In [48]:
mask = new_data['score'] > new_data['score_avg']
d = new_data[mask]
d

,student,exam1,exam2,exam3,score,total,min,better_than_average,score_avg
class,,,,,,,,,
Java,Alice,66,77,88,82.5,231,66,True,72.0
Java,Bob,89,53,56,72.5,198,53,True,72.0
Python,Bob,56,88,99,93.5,243,56,True,90.0
Python,Sam,89,99,88,94.0,276,88,True,90.0


In [49]:
#if we are only interested in student names
set(d.student)

{'Alice', 'Bob', 'Sam'}

# 4

In [42]:
#approach-1
# first determine whether student did better than average in each course
df['better_than_average']  = df.groupby('class')['score'].transform(lambda x: x > x.mean())
df

,class,student,exam1,exam2,exam3,score,total,min,better_than_average
0,Java,Alice,66,77,88,82.5,231,66,True
1,Python,Alice,56,77,88,82.5,221,56,False
2,Java,Bob,89,53,56,72.5,198,53,True
3,Python,Bob,56,88,99,93.5,243,56,True
4,Java,Bo,22,56,66,61.0,144,22,False
5,Python,Sam,89,99,88,94.0,276,88,True


In [30]:
# second, re-group per student, and count the number of courses
# that are better than average, the number should be equal to the
# total number of courses (e.g., 2 in this example)

In [44]:
total_courses = len(set(df['class']))
df2 = df.groupby('student').filter(lambda x: sum(
                x['better_than_average']
            )  == total_courses
        )
df2

,class,student,exam1,exam2,exam3,score,total,min,better_than_average
2,Java,Bob,89,53,56,72.5,198,53,True
3,Python,Bob,56,88,99,93.5,243,56,True


In [45]:
# if we are interested in student names
set(df2.student)

{'Bob'}

In [46]:
#good job Bob!

In [50]:
# approach-2
# start with student doing better in each course
d

,student,exam1,exam2,exam3,score,total,min,better_than_average,score_avg
class,,,,,,,,,
Java,Alice,66,77,88,82.5,231,66,True,72.0
Java,Bob,89,53,56,72.5,198,53,True,72.0
Python,Bob,56,88,99,93.5,243,56,True,90.0
Python,Sam,89,99,88,94.0,276,88,True,90.0


In [54]:
# retrieve students in each course
t = [set(d.loc[i].student) for i in set(d.index)]
t

[{'Bob', 'Sam'}, {'Alice', 'Bob'}]

In [55]:
# do set intersection
t[0] & t[1]

{'Bob'}

In [56]:
# in general if there are more than two courses
# reduce the list

In [57]:
from functools import reduce

In [58]:
reduce(lambda x, y: x & y, t)

{'Bob'}